# Opencorporates code snippets

The aim of this notebook is to create some code snippets to query the opencorporates database. These snippets can be reused in other code, or on other platforms such as recon-ng or maltego. 

Opencorporates does allow to query the API without a key up to approximately 100 times a day. They provide free keys for some open source projects. You can find the API reference of opencorporates [here](https://api.opencorporates.com/documentation/API-Reference) and more on API keys [here](https://opencorporates.com/api_accounts/new)

In [1]:
import requests
import pprint

First define a basic search function with pagination

In [2]:
def search_company(company_name, jurisdiction=None):
    
    '''search company by name and jurisdiction (optional)
    and returns a list of companies'''
    
    search_endpoint = 'https://api.opencorporates.com/v0.4/companies/search?q=' 

    if jurisdiction != None:
        query = search_endpoint + company_name + '&jurisdiction_code=' + jurisdiction
    
    else:
        query = search_endpoint + company_name
        
    result = requests.get(query).json()
    print(result)
    companies = []
    
    for company in result['results']['companies']:
        companies.append(company)
    
    total_pages = result['results'].get('total_pages')
    count = 2
    
    if total_pages > 1:
        while count <= total_pages:
            query2 = query + '&page=' + str(count)
            result = requests.get(query2).json()
            for company in result['results']['companies']:
                companies.append(company)

            count +=1
            
    return companies

see if it works...

In [3]:
nl = search_company('fox-it', jurisdiction='nl')
world = search_company('fox-it')

{'api_version': '0.4', 'results': {'companies': [{'company': {'name': 'David Fox IT', 'company_number': '67894321', 'jurisdiction_code': 'nl', 'incorporation_date': None, 'dissolution_date': None, 'company_type': None, 'registry_url': 'https://www.kvk.nl/zoeken/handelsregister/#!uitgebreid-zoeken&handelsnaam=&kvknummer=67894321&straat=&postcode=&huisnummer=&plaats=&hoofdvestiging=true&rechtspersoon=true&nevenvestiging=false&zoekvervallen=1&zoekuitgeschreven=1&start=0&initial=0&searchfield=uitgebreidzoeken', 'branch': None, 'branch_status': None, 'inactive': False, 'current_status': 'Active', 'created_at': '2017-06-07T20:50:22+00:00', 'updated_at': '2019-07-12T03:18:01+00:00', 'retrieved_at': '2019-07-09T06:24:32+00:00', 'opencorporates_url': 'https://opencorporates.com/companies/nl/67894321', 'previous_names': [], 'source': {'publisher': 'Kamer van Koophandel (KvK)', 'url': 'https://www.kvk.nl/zoeken/handelsregister/#!uitgebreid-zoeken&handelsnaam=&kvknummer=67894321&straat=&postcode=&

In [4]:
count = 0
for company in nl:
    count +=1
print('found ' + str(count) + ' companies')

pprint.pprint(nl)

found 5 companies
[{'company': {'branch': None,
              'branch_status': None,
              'company_number': '67894321',
              'company_type': None,
              'created_at': '2017-06-07T20:50:22+00:00',
              'current_status': 'Active',
              'dissolution_date': None,
              'inactive': False,
              'incorporation_date': None,
              'industry_codes': [],
              'jurisdiction_code': 'nl',
              'name': 'David Fox IT',
              'native_company_number': None,
              'opencorporates_url': 'https://opencorporates.com/companies/nl/67894321',
              'previous_names': [],
              'registered_address': None,
              'registered_address_in_full': None,
              'registry_url': 'https://www.kvk.nl/zoeken/handelsregister/#!uitgebreid-zoeken&handelsnaam=&kvknummer=67894321&straat=&postcode=&huisnummer=&plaats=&hoofdvestiging=true&rechtspersoon=true&nevenvestiging=false&zoekvervallen=1&zoekui

In [5]:
count = 0
for company in world:
    count +=1
print('found ' + str(count) + ' companies')

pprint.pprint(world)

found 99 companies
[{'company': {'branch': None,
              'branch_status': None,
              'company_number': '06678570',
              'company_type': 'Private Limited Company',
              'created_at': '2010-10-23T11:24:38+00:00',
              'current_status': 'Dissolved',
              'dissolution_date': '2012-09-04',
              'inactive': True,
              'incorporation_date': '2008-08-21',
              'industry_codes': [],
              'jurisdiction_code': 'gb',
              'name': '365 TECHNOLOGY LIMITED',
              'native_company_number': None,
              'opencorporates_url': 'https://opencorporates.com/companies/gb/06678570',
              'previous_names': [{'company_name': 'FOX IT HOLDING LIMITED',
                                  'end_date': '2012-01-05',
                                  'start_date': '2011-12-06'},
                                 {'company_name': '365 TECHNOLOGY LIMITED',
                                  'end_date': '2

              'opencorporates_url': 'https://opencorporates.com/companies/de/R1901_HRB9792',
              'previous_names': [{'company_name': 'Endless Fantasy UG '
                                                  '(haftungsbeschränkt)'},
                                 {'company_name': 'Silver Fox IT Services UG '
                                                  '(haftungsbeschränkt)'}],
              'registered_address': {'country': 'Germany',
                                     'locality': None,
                                     'postal_code': None,
                                     'region': None,
                                     'street_address': 'Hirtenstraße 64, 59823 '
                                                       'Arnsberg.'},
              'registered_address_in_full': 'Hirtenstraße 64, 59823 Arnsberg.',
              'registry_url': None,
              'restricted_for_marketing': None,
              'retrieved_at': '2018-06-24T02:53:21+00:00',
       

                                                    'description': 'Computer '
                                                                   'consultancy '
                                                                   'and '
                                                                   'computer '
                                                                   'facilities '
                                                                   'management '
                                                                   'activities',
                                                    'uid': 'isic_4-6202'}}],
              'jurisdiction_code': 'gb',
              'name': 'FLYING FOX I.T. LIMITED',
              'native_company_number': None,
              'opencorporates_url': 'https://opencorporates.com/companies/gb/03524309',
              'previous_names': [],
              'registered_address': {'country': 'United Kingdom',
                                     'loc

              'registry_url': 'http://www.business.govt.nz/companies/app/ui/pages/companies/1155980/detail',
              'restricted_for_marketing': None,
              'retrieved_at': '2017-06-14T05:45:33+00:00',
              'source': {'publisher': 'Ministry of Business, Innovation & '
                                      'Employment NZBI API',
                         'retrieved_at': '2017-06-14T05:45:33+00:00',
                         'url': 'https://api.business.govt.nz'},
              'updated_at': '2017-06-14T05:45:33+00:00'}},
 {'company': {'branch': None,
              'branch_status': None,
              'company_number': '392427',
              'company_type': 'LTD - Private Company Limited by Shares',
              'created_at': '2011-07-19T12:01:50+00:00',
              'current_status': 'Normal',
              'dissolution_date': None,
              'inactive': False,
              'incorporation_date': '2004-10-14',
              'industry_codes': [],
             

              'industry_codes': [{'industry_code': {'code': '62.02',
                                                    'code_scheme_id': 'uk_sic_2007',
                                                    'code_scheme_name': 'UK '
                                                                        'SIC '
                                                                        'Classification '
                                                                        '2007',
                                                    'description': 'Computer '
                                                                   'consultancy '
                                                                   'activities',
                                                    'uid': 'uk_sic_2007-6202'}},
                                 {'industry_code': {'code': '62.02',
                                                    'code_scheme_id': 'eu_nace_2',
                                            

 {'company': {'branch': 'F',
              'branch_status': 'branch of an out-of-jurisdiction company',
              'company_number': '602472254',
              'company_type': 'FOREIGN LIMITED LIABILITY COMPANY',
              'created_at': '2011-05-19T12:40:31+00:00',
              'current_status': 'Terminated',
              'dissolution_date': '2008-06-02',
              'inactive': True,
              'incorporation_date': '2005-02-01',
              'industry_codes': [],
              'jurisdiction_code': 'us_wa',
              'name': 'FOX IT, LLC',
              'native_company_number': None,
              'opencorporates_url': 'https://opencorporates.com/companies/us_wa/602472254',
              'previous_names': [],
              'registered_address': {'country': 'UNITED STATES',
                                     'locality': 'BRIDGEPORT',
                                     'postal_code': '19405',
                                     'region': 'PA',
                   

              'dissolution_date': None,
              'inactive': True,
              'incorporation_date': '2013-08-30',
              'industry_codes': [],
              'jurisdiction_code': 'us_fl',
              'name': 'RED FOX IT, LLC',
              'native_company_number': None,
              'opencorporates_url': 'https://opencorporates.com/companies/us_fl/L13000123458',
              'previous_names': [],
              'registered_address': None,
              'registered_address_in_full': None,
              'registry_url': None,
              'restricted_for_marketing': None,
              'retrieved_at': '2019-01-01T00:00:00+00:00',
              'source': {'publisher': 'Florida Department of State Division of '
                                      'Corporations',
                         'retrieved_at': '2019-01-01T00:00:00+00:00',
                         'url': 'http://www.sunbiz.org'},
              'updated_at': '2019-02-21T14:32:57+00:00'}},
 {'company': {'branch': 

Parse the data and extract relevant information, such as name, company_number, company_type, current_status, incorporation_date, dissolution_date, inactive, jurisdiction_code, opencorporates_url, previous_names, street_address, postal_code, country, address_in_full, source_url, source.

In [6]:
def parse_company_data(result):
    
    '''parses results of company search
    and returns list of tuples'''
    
    for company in result:
        name = company['company'].get('name', 'unknown')
        company_number = company['company'].get('company_number', 'unknown')
        company_type = company['company'].get('company_type', 'unknown')
        current_status = company['company'].get('current_status', 'unknown')
        incorporation_date = company['company'].get('incorporation_date', 'unknown')
        dissolution_date = company['company'].get('dissolution_date', 'still active')
        inactive = company['company'].get('inactive', 'unknown')
        jurisdiction_code = company['company'].get('jurisdiction_code', 'unknown.')
        opencorporates_url = company['company'].get('opencorporates_url', 'unknown')
        previous_names = company['company'].get('previous_names', 'no previous names')
        try:
            street_address = company['company']['registered_address'].get('street_address', 'unknown')
        except:
            street_address = 'unknown'
        try:
            postal_code = company['company']['registered_address'].get('postal_code', 'unknown')
        except:
            postal_code = 'unknown'
        try:
            country = company['company']['registered_address'].get('country', 'unknown')
        except:
            country = 'unknown'
        
        address_in_full = company['company'].get('registered_address_in_full', 'unknown')
        source_url = company['company'].get('registry_url', 'unknown')
        source = company['company']['source'].get('publisher', 'unknown')

        yield name, company_number, company_type, current_status, \
            incorporation_date, dissolution_date, inactive, jurisdiction_code, \
            opencorporates_url, previous_names, street_address, postal_code, \
            country, address_in_full, source_url, source
            

In [7]:
companies = list(parse_company_data(world))

In [8]:
for result in companies:
    name, company_number, company_type, current_status, \
    incorporation_date, dissolution_date, inactive, jurisdiction_code, \
    opencorporates_url, previous_names, street_address, postal_code, \
    country, address_in_full, source_url, source = result
    print(name, company_number, jurisdiction_code, inactive)

365 TECHNOLOGY LIMITED 06678570 gb True
ACE COMPUTER CONSULTING PTY LTD 093401408 au True
AGILE FOX IT PTY LTD 614865502 au False
ANDREW FOX IT LIMITED 11518742 gb False
AS FOX IT CONSULTING LIMITED 11929097 gb False
BLUE FOX IT CONSULTING LIMITED 12069880 gb False
Belarion GmbH R1901_HRB9792 de False
CLEAR FOX IT LIMITED 09718284 gb False
COMPUTACENTER (FMS) LIMITED 03798091 gb False
David Fox IT 67894321 nl False
F-O-X IT Corporation 1362303 us_nc False
FIX IT FOX HANDYMAN SERVICES LLC 4172490 us_oh False
FIX IT FOX PLUMBING LLC 0802828321 us_tx False
FIX IT FOX, LLC L18464147 us_az None
FIX-IT-FOX INC. 0801187083 us_tx True
FLYING FOX I.T. LIMITED 03524309 gb True
FLYING FOX IT MANAGEMENT LIMITED 08594116 gb False
FO-FOX IT, INC. 3947947 us_pa False
FOX DEN STORE-IT 51-11 LLC F052685 us_wi True
FOX DEN STORE-IT 51-11 LLC F052635 us_wi False
FOX DEN STORE-IT LLC F047038 us_wi False
FOX DEN STORE-IT MOON MEADOWS LLC DL047819 us_sd False
FOX DEN STORE-IT OF NEWVILLE LLC F051705 us_wi F

Get more detailed info on companies

In [9]:
def get_company_info(jurisdiction, company_number):
    
    '''gets comprehensive data on
    companies based on jurisdiction code
    and company number'''
    
    company_endpoint = 'https://api.opencorporates.com/v0.4/companies/'
    query = company_endpoint + jurisdiction + '/' + str(company_number)
    result = requests.get(query).json()
    
    return result

In [10]:
jurisdicion = 'gb'
company_number = 11929097

result = get_company_info(jurisdicion, company_number)

In [11]:
pprint.pprint(result)

{'api_version': '0.4',
 'results': {'company': {'agent_address': None,
                         'agent_name': None,
                         'alternate_registration_entities': [],
                         'alternative_names': [],
                         'branch': None,
                         'branch_status': None,
                         'company_number': '11929097',
                         'company_type': 'Private Limited Company',
                         'controlling_entity': None,
                         'corporate_groupings': [],
                         'created_at': '2019-04-05T13:13:16+00:00',
                         'current_status': 'Active',
                         'data': None,
                         'dissolution_date': None,
                         'filings': [{'filing': {'date': '2019-04-05',
                                                 'description': 'Incorporation',
                                                 'filing_type_code': 'NEWINC',
           

In [16]:
def get_officers(result):
    
    '''parses results of officer search
    and returns list of tuples'''
    
    for company in result['results']['company']['officers']:
        for officer in company:
            name = officer['officers'].get('name', 'unknown')
            current_state = officer['officer'].get('current_Status', 'unknown')
            officer_id = officer['officer'].get('id', 'unkown')
            inactive = officer['officer'].get('inactive', 'unknown')
            occupation = officer['officer'].get('occupation', 'unknown')
            opencorporates_url = officer['officer'].get('opencorporates_url', 'unknown')
            position = officer['officer'].get('position', 'unknown')
            start_date = officer['officer'].get('start_date', 'unknown')
            uid = officer['officer'].get('uid', 'unknown')
            try:
                end_date = officer['officer'].get('end_date', 'unknown')
            except:
                end_date = None
                pass
            
        yield name, current_state, officer_id, inactive, \
                occupation, opencorporates_url, position, \
                start_date, uid, end_date
            

In [17]:
for results in get_officers(result):
    name, current_state, officer_id, inactive, \
    occupation, opencorporates_url, position, \
    start_date, uid, end_date = results
    
print(name)

TypeError: string indices must be integers